# Setup (on collab)

In [ ]:
# Executa apenas se estiver rodando no Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/alicebarbosam/fraud-detection-kaggle.git
    %cd fraud-detection-kaggle
    !pip install -r requirements-colab.txt

In [30]:
import os
import joblib
import pandas as pd

model = joblib.load("../models/xgb_optuna_final.joblib")

test_df = pd.read_csv("../data/test.csv")

X_test = test_df.drop(columns=["id"])
test_ids = test_df["id"]

test_probs = model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    "id": test_ids,
    "target": test_probs
})

os.makedirs("../outputs", exist_ok=True)
submission.to_csv("../outputs/submission.csv", index=False)

print("Saved: ../outputs/submission.csv")
print(submission.head())

Saved: ../outputs/submission.csv
       id    target
0  263021  0.000147
1   11379  0.000130
2  147284  0.002449
3  219440  0.000074
4   36940  0.010155


In [31]:
print(submission["target"].describe())
print("max:", submission["target"].max())
print("min:", submission["target"].min())
print("count > 0.5:", (submission["target"] > 0.5).sum())
print("count > 0.1:", (submission["target"] > 0.1).sum())
print("count > 0.01:", (submission["target"] > 0.01).sum())

count    56962.000000
mean         0.004060
std          0.046673
min          0.000032
25%          0.000174
50%          0.000318
75%          0.000762
max          0.999825
Name: target, dtype: float64
max: 0.9998246
min: 3.167736e-05
count > 0.5: 132
count > 0.1: 286
count > 0.01: 1678
